In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("./data/stock-market-data.csv")

1. 哪些股票的代码中包含"8"这个数字？

In [6]:
data[data["symbol"].str.contains("8")]["symbol"].unique()[:5]

array(['600008.SH', '600018.SH', '600028.SH', '600038.SH', '600048.SH'],
      dtype=object)

2.  每天上涨和下跌的股票各有多少？

In [3]:
data["tag"] = np.where(data["close"] - data["pre_close"] > 0, "上涨", 
                       np.where(data["close"] - data["pre_close"] < 0, "下跌", "不变"))
data.groupby(['date', 'tag'])['symbol'].count()

date      tag
20120104  上涨      191
          下跌     2007
          不变      122
20120105  上涨      132
          下跌     2071
                 ... 
20120628  下跌     1819
          不变      176
20120629  上涨     1890
          下跌      358
          不变      174
Name: symbol, Length: 351, dtype: int64

3. 每天每个交易所上涨、下跌的股票各有多少？

In [4]:
data["tag"] = np.where(data["close"] - data["pre_close"] > 0, "上涨", 
                       np.where(data["close"] - data["pre_close"] < 0, "下跌", "不变"))
data["exchange"] = data['symbol'].str.strip().str[-2:]

data.groupby(["date", "exchange"])["symbol"].count()

date      exchange
20120104  SH           921
          SZ          1399
20120105  SH           921
          SZ          1399
20120106  SH           921
                      ... 
20120627  SZ          1484
20120628  SH           935
          SZ          1487
20120629  SH           935
          SZ          1487
Name: symbol, Length: 234, dtype: int64

4. 沪深300成分股中，每天上涨、下跌的股票各有多少？

In [5]:
data["tag"] = np.where(data["close"] - data["pre_close"] > 0, "上涨", 
                       np.where(data["close"] - data["pre_close"] < 0, "下跌", "不变"))
data[data.index_w300 > 0].groupby(['date', 'tag'])["symbol"].count()

date      tag
20120104  上涨      20
          下跌     275
          不变       5
20120105  上涨      50
          下跌     242
                ... 
20120628  下跌     229
          不变      19
20120629  上涨     244
          下跌      40
          不变      16
Name: symbol, Length: 350, dtype: int64

5. 每天每个行业各有多少只股票？

In [7]:
data.groupby(["date", "industry"])["symbol"].count()

date      industry
20120104  AERODEF      10
          AIRLINE      12
          AUTO         85
          BANKS        16
          BEV          30
                     ... 
20120629  REALEST     132
          RETAIL       74
          SOFTWARE     89
          TRDDIST      36
          UTILITIE     75
Name: symbol, Length: 3744, dtype: int64

6. 股票数最大的行业和总成交额最大的行业是否总是同一个行业？

In [35]:
data_industry = pd.DataFrame({"amount" : data.groupby(["date", "industry"])["amount"].sum(), 
                              "stknum" : data.groupby(["date", "industry"])["symbol"].size()}
                              ).reset_index()
data_industry["amount_seq"] = data_industry.sort_values(by = ["date", "amount"], ascending = [True, False]).groupby("date").cumcount().add(1)
data_industry["stknum_seq"] = data_industry.sort_values(by = ["date", "stknum"], ascending = [True, False]).groupby("date").cumcount().add(1)
data_industry[(data_industry['amount_seq'] == 1 )& (data_industry['stknum_seq'] == 1)]

,date,industry,amount,stknum,amount_seq,stknum_seq
15,20120104,HDWRSEMI,6.878138e+09,224,1,1
79,20120106,HDWRSEMI,5.979259e+09,224,1,1
751,20120213,HDWRSEMI,1.460269e+10,227,1,1
847,20120216,HDWRSEMI,1.506463e+10,227,1,1
879,20120217,HDWRSEMI,1.145135e+10,229,1,1
911,20120220,HDWRSEMI,1.525181e+10,230,1,1
943,20120221,HDWRSEMI,1.549184e+10,230,1,1
975,20120222,HDWRSEMI,1.990051e+10,230,1,1
1167,20120301,HDWRSEMI,1.486525e+10,231,1,1
1199,20120302,HDWRSEMI,1.620855e+10,231,1,1


7. 每天涨幅超过5%、跌幅超过5%的股票各有多少？

In [40]:
data["ret"] = data["close"]/data["pre_close"] - 1
data["tag"] = np.where(data["ret"] > 0.05, "up5%+",
                       np.where(data["ret"] < 0.05, "down5%+", "eq5%"))
data[data["tag"].str.contains("up5%+|down5%+")].groupby(["date", "tag"])["symbol"].size()

date      tag    
20120104  down5%+    2303
          up5%+        17
20120105  down5%+    2310
          up5%+        10
20120106  down5%+    2271
                     ... 
20120627  up5%+        63
20120628  down5%+    2396
          up5%+        26
20120629  down5%+    2349
          up5%+        73
Name: symbol, Length: 234, dtype: int64

8. 每天涨幅前10的股票的总成交额和跌幅前10的股票的总成交额比例是多少？

In [23]:
data["ret"] = data["close"]/data["pre_close"] - 1
data_top10amount = data.sort_values(by = ["date", "ret"]).groupby("date").head(10)
data_bottom10amount = data.sort_values(by = ["date", "ret"]).groupby("date").tail(10)
data_compare = pd.DataFrame({'top10amount':data_top10amount.groupby("date")["amount"].sum(),
                             'bottom10amount':data_bottom10amount.groupby("date")["amount"].sum()}).reset_index()
data_compare["ratio"] = data_compare['top10amount']/data_compare['bottom10amount']
data_compare.iloc[:5, [0, 3]]

,date,ratio
0,20120104,0.691766
1,20120105,0.580306
2,20120106,0.742506
3,20120109,0.351822
4,20120110,1.778087


9. 每天开盘涨停的股票与收盘涨停的股票各有多少？

In [24]:
data["tag"] = np.where((data["open"]/data["pre_close"] - 1) > 0.015, "openlimit", 
            np.where((data["close"]/data["pre_close"] - 1) > 0.015,     "closelimit", "others"
            ))
data[data["tag"].str.contains("openlimit|closelimit")].groupby(["date", "tag"])["symbol"].size()

date      tag       
20120104  closelimit     42
          openlimit     325
20120105  closelimit     55
          openlimit      27
20120106  closelimit    716
                       ... 
20120627  openlimit      37
20120628  closelimit    141
          openlimit      75
20120629  closelimit    944
          openlimit      22
Name: symbol, Length: 234, dtype: int64

10. 每天统计最近3天出现过开盘涨停的股票各有多少只？

In [4]:
data["tag"] = np.where((data["open"]/data["pre_close"] - 1) > 0.015, 1,0)
data_rolling = data.groupby("symbol").rolling(3, on = "date")["tag"].sum().reset_index()
data_rolling["tag"] = np.where(data_rolling["tag"] > 1, 1, data_rolling["tag"])
data_rolling.groupby("date").tag.sum().reset_index()

,date,tag
0,20120104,0.0
1,20120105,0.0
2,20120106,394.0
3,20120109,143.0
4,20120110,207.0
...,...,...
112,20120625,88.0
113,20120626,70.0
114,20120627,79.0
115,20120628,116.0


11. 股票每天的成交额变化率和收益率的相关性如何？

In [13]:
data['amount_lag'] = data.groupby("symbol")['amount'].shift(periods = 1)
data['amount_change'] = data['amount']/data['amount_lag'] - 1
data['ret'] = data['close']/data['pre_close'] - 1
data[~data.isin([np.nan, np.inf, -np.inf])]['amount_change'].corr(data[~data.isin([np.nan, np.inf, -np.inf])]['ret'])

0.22096889206014472

12. 每天每个行业的总成交额变化率和行业收益率的相关性如何？

In [16]:
data["ret"] = data["close"] - data["pre_close"]
data_ind = pd.DataFrame({'amount_industry': data.groupby(["date", "industry"])["amount"].sum(), 
            'ret_industry': data.groupby(["date", "industry"])["ret"].mean()}).reset_index().sort_values(by = ["industry", "date"])
data_ind['amount_industry_lag'] = data_ind.groupby("industry")['amount_industry'].shift(periods = 1)
data_ind["amount_change_industry"] = data_ind["amount_industry"]/data_ind["amount_industry_lag"] - 1
data_ind = data_ind.dropna(subset = ['amount_change_industry'])
data_ind.groupby(['date'])[["amount_change_industry", "ret_industry"]].corr()

amount_change_industry  ret_industry
date                                                                 
20120105 amount_change_industry                1.000000      0.313861
         ret_industry                          0.313861      1.000000
20120106 amount_change_industry                1.000000      0.082021
         ret_industry                          0.082021      1.000000
20120109 amount_change_industry                1.000000      0.114235
...                                                 ...           ...
20120627 ret_industry                          0.260895      1.000000
20120628 amount_change_industry                1.000000      0.168723
         ret_industry                          0.168723      1.000000
20120629 amount_change_industry                1.000000      0.606476
         ret_industry                          0.606476      1.000000

[232 rows x 2 columns]

13. 每天市场的总成交额变化率和市场收益率相关性如何？

In [35]:
data['ret'] = data['close']/data['pre_close'] - 1
# 生成关于成交量数据集
amount = data.groupby("date")["amount"].sum().reset_index().rename(columns = {"amount":"amount_mkt"})
amount["amount_mkt_lag"] = amount["amount_mkt"].shift(periods = 1)
amount["amount_mkt_change"] = amount["amount_mkt"]/amount["amount_mkt_lag"] - 1
amount = amount[~amount.isin([np.nan])]

# 生成关于收益率数据集
data_1 = data.merge(data.groupby("date")["capt"].sum().reset_index().rename(columns = {"capt":"capt_sum"}), left_on = "date", right_on="date")
data_1["capt_ratio"] = data_1["capt"]/data_1["capt_sum"]
data_1["ret_mkt"] = data_1["ret"]*data_1["capt_ratio"]
ret = data_1.groupby("date")["ret_mkt"].sum().reset_index()
ret.merge(amount, left_on = "date", right_on = "date")[["ret_mkt", "amount_mkt_change"]].corr()

,ret_mkt,amount_mkt_change
ret_mkt,1.000000,0.442459
amount_mkt_change,0.442459,1.000000


14. 每天市场的总成交额的变化率和所有股票收益率的标准差相关性如何？ 

In [14]:
data["ret"] = data["close"]/data["pre_close"] - 1
ret = data.groupby("date")['ret'].std().reset_index().rename(columns =  {"ret" : "ret_std"})

amount = data.groupby('date')['amount'].sum().reset_index().rename(columns = {"amount" : "amount_sum"})
amount["amount_sum_lag"] = amount["amount_sum"].shift(periods = 1)
amount["amount_change"] = amount["amount_sum"]/amount['amount_sum_lag'] - 1
amount = amount[~amount.isin([np.nan])]

ret.merge(amount, left_on = "date", right_on = "date")[['ret_std', 'amount_change']].corr()

,ret_std,amount_change
ret_std,1.000000,-0.191389
amount_change,-0.191389,1.000000


15. 每天每个行业的总成交额变化率和行业内股票收益率的标准差相关性如何？

In [39]:
data_15 = data
data_15["ret"] = data_15["close"]/data_15["pre_close"] - 1
ret = data_15.groupby(["date", "industry"])['ret'].std().reset_index().rename(columns = {"ret":"ret_std"})

amount = data_15.groupby(["industry", "date"])['amount'].sum().reset_index().rename(columns = {"amount":"amount_industry"})
amount = amount.merge(amount.groupby("industry")["amount_industry"].shift(periods=1).reset_index().rename(columns={"amount_industry":"amount_industry_lag"}), left_index=True, right_index=True)
amount['amount_industry_change'] = amount['amount_industry']/amount['amount_industry_lag'] - 1
ret_amount = ret.merge(amount, left_on = ["industry", "date"], right_on = ["industry", "date"])
# ret_amount = ret_amount[~ret_amount["amount_industry_change"].isin([np.nan])]
ret_amount.groupby("date")[["ret_std", "amount_industry_change"]].corr()


ret_std  amount_industry_change
date                                                             
20120104 ret_std                 1.000000                     NaN
         amount_industry_change       NaN                     NaN
20120105 ret_std                 1.000000               -0.366726
         amount_industry_change -0.366726                1.000000
20120106 ret_std                 1.000000                0.352095
...                                   ...                     ...
20120627 amount_industry_change  0.561366                1.000000
20120628 ret_std                 1.000000                0.089067
         amount_industry_change  0.089067                1.000000
20120629 ret_std                 1.000000               -0.032249
         amount_industry_change -0.032249                1.000000

[234 rows x 2 columns]

16. 上证50、沪深300、中证500指数成分股中，沪股和深股各有多少？

In [16]:
d_16 = data[(data.index_w50 > 0)|(data.index_w300 > 0)|(data.index_w500 > 0)]

d_16 = pd.concat([
    pd.DataFrame({
        "symbol": d_16[d_16.index_w50 > 0]["symbol"].unique().tolist(),
        "index" : "上证50"}),
    pd.DataFrame({
        "symbol":d_16[d_16.index_w300 > 0]["symbol"].unique().tolist(),
        "index" : "沪深300"}),
    pd.DataFrame({
        "symbol":d_16[d_16.index_w500 > 0]["symbol"].unique().tolist(),
        "index" : "中证500"
    })
    ])
d_16["tag"] = np.where(d_16["symbol"].str.contains("SH"), "沪股", "深股")
d_16.groupby(["index", "tag"])["symbol"].size()

index  tag
上证50   沪股      54
中证500  沪股     297
       深股     248
沪深300  沪股     205
       深股      95
Name: symbol, dtype: int64

17. 上证50、沪深300、中证500指数成分股中，行业分布如何？

In [40]:
d_17 = pd.concat([
    pd.DataFrame({
        "index" : "上证50",
        "industry" : data[data.index_w50 > 0].groupby("symbol")["industry"].first()
        }).reset_index(),
    pd.DataFrame({
        "index" : "沪深300",
        "industry" : data[data.index_w300 > 0].groupby("symbol")["industry"].first()
        }).reset_index(),
    pd.DataFrame({
        "index" : "中证500",
        "industry" : data[data.index_w500 > 0].groupby("symbol")["industry"].first()
        }).reset_index()
    ])
# d_17["industry"] = pd.Series(d_17["industry"])
d_17.groupby(["index", "industry"])["symbol"].size()

index  industry
上证50   AIRLINE      1
       AUTO         1
       BANKS       11
       BEV          1
       CHEM         1
                   ..
沪深300  REALEST     13
       RETAIL       7
       SOFTWARE     5
       TRDDIST      5
       UTILITIE    12
Name: symbol, Length: 79, dtype: int64

18. 每天上证50、沪深300、中证500指数成分股的总成交额各是多少

In [5]:
d_18 = pd.concat([
    pd.DataFrame({
        "index" : "上证50",
        "amount" : data[data.index_w50 > 0].groupby(["symbol", "date"])["amount"].first()
        }).reset_index(),
    pd.DataFrame({
        "index" : "沪深300",
        "amount" : data[data.index_w300 > 0].groupby(["symbol", "date"])["amount"].first()
        }).reset_index(),
    pd.DataFrame({
        "index" : "中证500",
        "amount" : data[data.index_w500 > 0].groupby(["symbol", "date"])["amount"].first()
        }).reset_index()
])
d_18.groupby(["index", "date"])["amount"].sum()

index  date    
上证50   20120104    1.137204e+10
       20120105    1.517460e+10
       20120106    1.145797e+10
       20120109    1.978309e+10
       20120110    2.584061e+10
                       ...     
沪深300  20120625    4.250351e+10
       20120626    3.389651e+10
       20120627    3.143760e+10
       20120628    3.441879e+10
       20120629    4.097654e+10
Name: amount, Length: 351, dtype: float64

19. 上证50、沪深300、中证500指数日收益率的历史波动率是多少？

In [11]:
data["ret"] = data["close"]/data["pre_close"] - 1

d_19 = pd.concat([
    pd.DataFrame(
        {
            "index" : "上证50",
            "ratio" : data[data.index_w50 > 0].groupby(["date", "symbol"])['index_w50'].first(),
            "ret" : data[data.index_w50 > 0].groupby(["date", "symbol"])['ret'].first()
        }
    ).reset_index(),
    pd.DataFrame(
        {
            "index" : "沪深300",
            "ratio" : data[data.index_w300 > 0]. groupby(["date", "symbol"])['index_w300'].first(),
            "ret" : data[data.index_w300 > 0].groupby(["date", "symbol"])['ret'].first()
        }
    ).reset_index(),
    pd.DataFrame(
        {
            "index" : "中证500",
            "ratio" : data[data.index_w500 > 0]. groupby(["date", "symbol"])['index_w500'].first(),
            "ret" : data[data.index_w500 > 0].groupby(["date", "symbol"])['ret'].first()
        }
    ).reset_index(),    
])
d_19["ratio_ret"] = d_19["ret"] * d_19["ratio"]
d_19 = d_19.groupby(["index", "date"])["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret":"index_ret"})
d_19.groupby("index")["index_ret"].std()

index
上证50     0.012342
中证500    0.016075
沪深300    0.013368
Name: index_ret, dtype: float64

20. 上证50、沪深300、中证500指数日收益率的相关系数矩阵？

In [16]:
data["ret"] = data["close"]/data["pre_close"] - 1

d_20 = pd.concat([
    pd.DataFrame(
        {
            "index" : "上证50",
            "ratio" : data[data.index_w50 > 0].groupby(["date", "symbol"])['index_w50'].first(),
            "ret" : data[data.index_w50 > 0].groupby(["date", "symbol"])['ret'].first()
        }
    ).reset_index(),
    pd.DataFrame(
        {
            "index" : "沪深300",
            "ratio" : data[data.index_w300 > 0]. groupby(["date", "symbol"])['index_w300'].first(),
            "ret" : data[data.index_w300 > 0].groupby(["date", "symbol"])['ret'].first()
        }
    ).reset_index(),
    pd.DataFrame(
        {
            "index" : "中证500",
            "ratio" : data[data.index_w500 > 0]. groupby(["date", "symbol"])['index_w500'].first(),
            "ret" : data[data.index_w500 > 0].groupby(["date", "symbol"])['ret'].first()
        }
    ).reset_index(),    
])
d_20["ratio_ret"] = d_20["ret"] * d_20["ratio"]
d_20 = d_20.groupby(["index", "date"])["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret":"index_ret"})
d_20 = d_20.pivot_table(index = 'date', columns = 'index', values = "index_ret").reset_index()
d_20[["上证50", "沪深300", "中证500"]].corr()

index,上证50,沪深300,中证500
index,,,
上证50,1.000000,0.976085,0.847698
沪深300,0.976085,1.000000,0.933358
中证500,0.847698,0.933358,1.000000


21. 上证50、沪深300、去除上证50的沪深300指数日收益率的相关系数矩阵？

In [4]:
data["ret"] = data["close"]/data["pre_close"] - 1
d_21 = pd.concat([
    pd.DataFrame(
        {
            "index" : "上证50",
            "ratio" : data[data.index_w50 > 0].groupby(["date", "symbol"])['index_w50'].first(),
            "ret" : data[data.index_w50 > 0].groupby(["date", "symbol"])['ret'].first()
        }
    ).reset_index(),
    pd.DataFrame(
        {
            "index" : "沪深300",
            "ratio" : data[data.index_w300 > 0]. groupby(["date", "symbol"])['index_w300'].first(),
            "ret" : data[data.index_w300 > 0].groupby(["date", "symbol"])['ret'].first()
        }
    ).reset_index(),
    pd.DataFrame(
        {
            "index" : "沪深300去除上证50",
            "ratio" : data[(data.index_w300 > 0) & (data.index_w50 == 0)]. groupby(["date", "symbol"])['index_w300'].first(),
            "ret" : data[(data.index_w300 > 0) & (data.index_w50 == 0)].groupby(["date", "symbol"])['ret'].first()
        }
    ).reset_index(),    
])
d_21["ratio_ret"] = d_21["ret"] * d_21["ratio"]
d_21 = d_21.groupby(["index", "date"])["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret":"index_ret"})
d_21 = d_21.pivot_table(index = 'date', columns = 'index', values = "index_ret").reset_index()
d_21[["上证50", "沪深300", "沪深300去除上证50"]].corr()

index,上证50,沪深300,沪深300去除上证50
index,,,
上证50,1.000000,0.976085,0.929438
沪深300,0.976085,1.000000,0.987405
沪深300去除上证50,0.929438,0.987405,1.000000


22. 每天沪深300指数成分占比最大的10只股票是哪些？

In [13]:
data[data.index_w300 > 0].sort_values(by = ["date", "index_w300"], ascending = False).groupby("date")[['symbol', 'date']].head(10)

,symbol,date
99884,601318.SH,20120629
1403,600016.SH,20120629
3158,600036.SH,20120629
100001,601328.SH,20120629
49679,600519.SH,20120629
...,...,...
0,600000.SH,20120104
95531,601088.SH,20120104
49563,600519.SH,20120104
108533,000002.SZ,20120104


23. 各个行业的平均每日股票数量从大到小排序是什么？

In [16]:
d_23 = data.groupby(["industry", "date"])["symbol"].count().reset_index()
d_23.sort_values(by = ["date", "symbol"], ascending = False)

,industry,date,symbol
1871,HDWRSEMI,20120629,242
818,CHEM,20120629,225
2456,MACH,20120629,208
1988,HEALTH,20120629,183
1520,ELECEQP,20120629,138
...,...,...,...
351,BANKS,20120104,16
2106,INDCONG,20120104,13
117,AIRLINE,20120104,12
2457,MARINE,20120104,12


24. 每个行业每天成交额最大的一只股票代码是什么？

In [24]:
data.sort_values(by = ["industry", "date", "amount"], ascending = False).groupby(["industry", "date"])['symbol'].first().reset_index()

,industry,date,symbol
0,AERODEF,20120104,000768.SZ
1,AERODEF,20120105,000768.SZ
2,AERODEF,20120106,000768.SZ
3,AERODEF,20120109,600316.SH
4,AERODEF,20120110,000768.SZ
...,...,...,...
3739,UTILITIE,20120625,600011.SH
3740,UTILITIE,20120626,300262.SZ
3741,UTILITIE,20120627,600011.SH
3742,UTILITIE,20120628,600011.SH


25. 每个行业每天最大成交额是最小成交额的几倍？

In [35]:
data.groupby(["industry", "date"])["amount"].agg(np.ptp).reset_index()


,industry,date,amount
0,AERODEF,20120104,159432638.0
1,AERODEF,20120105,129651459.1
2,AERODEF,20120106,184793300.7
3,AERODEF,20120109,90420423.0
4,AERODEF,20120110,152842538.4
...,...,...,...
3739,UTILITIE,20120625,358829878.0
3740,UTILITIE,20120626,193623101.5
3741,UTILITIE,20120627,172117879.0
3742,UTILITIE,20120628,217489778.0


26. 每个行业每天成交额最大的5只股票的成交额总和是多少？

In [42]:
d_26 = data.sort_values(by = ["industry", "date", "amount"], ascending = False).groupby(['industry', 'date']).head(5)
d_26.groupby(["industry", "date"])['amount'].sum().reset_index()

,industry,date,amount
0,AERODEF,20120104,3.847582e+08
1,AERODEF,20120105,4.422500e+08
2,AERODEF,20120106,5.428553e+08
3,AERODEF,20120109,3.834715e+08
4,AERODEF,20120110,6.540624e+08
...,...,...,...
3739,UTILITIE,20120625,1.158774e+09
3740,UTILITIE,20120626,6.363266e+08
3741,UTILITIE,20120627,7.098671e+08
3742,UTILITIE,20120628,7.635796e+08


27. 每个行业每天成交额超过该行业中股票成交额80%分位数的股票的平均收益率是多少？

In [50]:
data["ret"] = data["close"]/data["pre_close"] - 1
data[data.amount > data.groupby(["industry", "date"]).amount.transform('quantile', 0.8)].groupby(["industry", "date"])["ret"].mean().reset_index()


,industry,date,ret
0,AERODEF,20120104,0.005714
1,AERODEF,20120105,0.006090
2,AERODEF,20120106,-0.050832
3,AERODEF,20120109,0.022119
4,AERODEF,20120110,0.053002
...,...,...,...
3739,UTILITIE,20120625,-0.002189
3740,UTILITIE,20120626,-0.008984
3741,UTILITIE,20120627,0.003968
3742,UTILITIE,20120628,-0.006003


28. 每天成交额最大的10%的股票的平均收益率和成交额最小的10%的股票的平均收益率的相关系数是多少？

In [57]:
data["ret"] = data["close"]/data["pre_close"] - 1
d_28 = pd.DataFrame({
    "amount_max": data[data.amount >= data.groupby(["date"]).amount.transform('quantile', 0.9)].groupby(["industry", "date"])["ret"].mean(),
    "amount_min": data[data.amount <= data.groupby(["date"]).amount.transform('quantile', 0.1)].groupby(["industry", "date"])["ret"].mean()
}).reset_index()
d_28["amount_min"].corr(d_28["amount_max"])

0.3658878486773265

29. 每天哪些行业的平均成交额高于全市场平均成交额？

In [65]:
d_29 = pd.merge(
    data.groupby(["industry", "date"]).amount.mean().reset_index(),
    data.groupby("date").amount.mean().reset_index(),
    on = 'date'
)
d_29[d_29.amount_x > d_29.amount_y][["industry", "date"]].reset_index()

,index,industry,date
0,0,AERODEF,20120104
1,3,BANKS,20120104
2,4,BEV,20120104
3,9,CONMAT,20120104
4,11,DVFININS,20120104
...,...,...,...
1408,3730,INDCONG,20120629
1409,3733,MARINE,20120629
1410,3735,MEDIA,20120629
1411,3736,MTLMIN,20120629


30. 每天每个股票对市场的超额收益率是多少？

In [22]:
# data cleaning
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1
d_30 = data.groupby("date")["capt"].sum().reset_index().rename(columns = {"capt": "mkt_capt"})
d_30 = data.merge(d_30, left_on = "date", right_on = "date")
d_30["capt_ratio"] = d_30["capt"]/d_30["mkt_capt"]
d_30["ratio_ret"] = d_30["capt_ratio"]*d_30["stkcd_ret"]
d_30 = d_30.groupby("date")["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret": "mkt_ret"})
d_30 = data[["date", "symbol", "stkcd_ret"]].merge(d_30, right_on = "date", left_on = "date")

# linear model OLS
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params
d_30 = d_30.groupby("symbol").apply(regress, 'stkcd_ret', ['mkt_ret']).reset_index().rename(columns = {"mkt_ret":"beta"}).merge(d_30, right_on = "symbol", left_on = "symbol")
d_30['abnr_ret'] = d_30["stkcd_ret"] - d_30["intercept"] - d_30["mkt_ret"]*d_30["beta"]
d_30[["symbol", "date", "abnr_ret"]]

,symbol,date,abnr_ret
0,000001.SZ,20120104,-0.015698
1,000001.SZ,20120105,0.026020
2,000001.SZ,20120106,-0.006935
3,000001.SZ,20120109,0.006368
4,000001.SZ,20120110,-0.020913
...,...,...,...
276895,603366.SH,20120625,-0.033429
276896,603366.SH,20120626,-0.017837
276897,603366.SH,20120627,0.004242
276898,603366.SH,20120628,-0.005762


31. 每天每个股票对市场去除自身的超额收益率是多少？

In [17]:
# data cleaning
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1
d_31 = data
d_31_group = d_31.groupby("date")
d_31_list = []
for name, group in d_31_group:
    for i in range(0, len(group)):
        a = group.drop(group.index[i])["capt"]/(group.drop(group.index[i])["capt"].sum()).tolist()
        b = group.drop(group.index[i])["stkcd_ret"].tolist()
        # print(b[:5])
        d_31_list.append(sum(a*b))
d_31["mkt_ret"] = d_31_list

# linear model OLS
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params
d_31 = d_31.groupby("symbol").apply(regress, 'stkcd_ret', ['mkt_ret']).reset_index().rename(columns = {"mkt_ret":"beta"}).merge(d_31, right_on = "symbol", left_on = "symbol")
d_31['abnr_ret'] = d_31["stkcd_ret"] - d_31["intercept"] - d_31["mkt_ret"]*d_31["beta"]
d_31[["symbol", "date", "abnr_ret"]]


,symbol,date,abnr_ret
0,000001.SZ,20120104,-0.027643
1,000001.SZ,20120105,0.015143
2,000001.SZ,20120106,-0.002595
3,000001.SZ,20120109,0.028236
4,000001.SZ,20120110,0.000082
...,...,...,...
276895,603366.SH,20120625,-0.052794
276896,603366.SH,20120626,-0.018613
276897,603366.SH,20120627,0.002372
276898,603366.SH,20120628,-0.014576


In [113]:
d = pd.DataFrame({"A":[1, 2, 3, 4, 5, 6], "B":["a", 'a', 'a', 'b', 'b', 'b'], "C":[6, 5, 4, 3, 2, 1]})
d_group =d.groupby("B")
list = []
# for name, group in d_group:
for name, group in d_group:
    for i in range(len(group)):
        a = group.drop(group.index[i])["A"]/(group.drop(group.index[i])["A"].sum()).tolist()
        b = group.drop(group.index[i])["C"].tolist()
        list.append(sum(a*b))
    # print(d["C"])
print(list)
d["D"] = list
# d

[4.4, 4.5, 5.333333333333333, 1.4545454545454546, 1.8000000000000003, 2.4444444444444446]


32. 每天每个股票对行业的超额收益率是多少？

In [12]:
# data cleaning
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1
d_32 = data.groupby(["date", "industry"])["capt"].sum().reset_index().rename(columns = {"capt": "industry_capt"})
d_32 = data.merge(d_32, left_on = ["date", "industry"], right_on = ["date", "industry"])
d_32["capt_ratio"] = d_32["capt"]/d_32["industry_capt"]
d_32["ratio_ret"] = d_32["capt_ratio"]*d_32["stkcd_ret"]
d_32 = d_32.groupby(["date", "industry"])["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret": "industry_ret"})
d_32 = data[["date", "symbol", "stkcd_ret"]].merge(d_32, right_on = "date", left_on = "date")


# linear model OLS
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params
d_32 = d_32.groupby("symbol").apply(regress, 'stkcd_ret', ['industry_ret']).reset_index().rename(columns = {"industry_ret":"beta"}).merge(d_32, right_on = "symbol", left_on = "symbol")
d_32['abnr_ret'] = d_32["stkcd_ret"] - d_32["intercept"] - d_32["industry_ret"]*d_32["beta"]
d_32[["symbol", "date", "abnr_ret"]]

,symbol,date,abnr_ret
0,000001.SZ,20120104,-0.023570
1,000001.SZ,20120104,-0.019238
2,000001.SZ,20120104,-0.021616
3,000001.SZ,20120104,-0.024270
4,000001.SZ,20120104,-0.009228
...,...,...,...
8860795,603366.SH,20120629,0.017301
8860796,603366.SH,20120629,0.015986
8860797,603366.SH,20120629,0.012686
8860798,603366.SH,20120629,0.018352


33. 每天每个股票对行业去除自身的超额收益率是多少？

In [10]:
# data cleaning
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1
d_33 = data
d_33_group = d_33.groupby(["date", "industry"])
d_33_list = []
for name, group in d_33_group:
    for i in range(len(group)):
        a = group.drop(group.index[i])["capt"]/(group.drop(group.index[i])["capt"].sum()).tolist()
        b = group.drop(group.index[i])["stkcd_ret"].tolist()
        # print(b[:5])
        d_33_list.append(sum(a*b))
d_33["industry_ret"] = d_33_list

# linear model OLS
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params
d_33 = d_33.groupby("symbol").apply(regress, 'stkcd_ret', ['industry_ret']).reset_index().rename(columns = {"industry_ret":"beta"}).merge(d_33, right_on = "symbol", left_on = "symbol")
d_33['abnr_ret'] = d_33["stkcd_ret"] - d_33["intercept"] - d_33["industry_ret"]*d_33["beta"]
d_33[["symbol", "date", "abnr_ret"]]

,symbol,date,abnr_ret
0,000001.SZ,20120104,-0.027896
1,000001.SZ,20120105,0.014859
2,000001.SZ,20120106,-0.002913
3,000001.SZ,20120109,0.027698
4,000001.SZ,20120110,-0.000314
...,...,...,...
276895,603366.SH,20120625,-0.039445
276896,603366.SH,20120626,-0.018809
276897,603366.SH,20120627,0.004924
276898,603366.SH,20120628,-0.013138


34. 每个股票每天对市场的超额收益率与对行业的超额收益率的相关系数如何？

In [17]:
# 对市场超额收益率
# data cleaning
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1
d_30 = data.groupby("date")["capt"].sum().reset_index().rename(columns = {"capt": "mkt_capt"})
d_30 = data.merge(d_30, left_on = "date", right_on = "date")
d_30["capt_ratio"] = d_30["capt"]/d_30["mkt_capt"]
d_30["ratio_ret"] = d_30["capt_ratio"]*d_30["stkcd_ret"]
d_30 = d_30.groupby("date")["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret": "mkt_ret"})
d_30 = data[["date", "symbol", "stkcd_ret"]].merge(d_30, right_on = "date", left_on = "date")

# linear model OLS
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params
d_30 = d_30.groupby("symbol").apply(regress, 'stkcd_ret', ['mkt_ret']).reset_index().rename(columns = {"mkt_ret":"beta"}).merge(d_30, right_on = "symbol", left_on = "symbol")
d_30['abnr_ret'] = d_30["stkcd_ret"] - d_30["intercept"] - d_30["mkt_ret"]*d_30["beta"]

# 对行业超额收益率
# data cleaning
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1
d_32 = data.groupby(["date", "industry"])["capt"].sum().reset_index().rename(columns = {"capt": "industry_capt"})
d_32 = data.merge(d_32, left_on = ["date", "industry"], right_on = ["date", "industry"])
d_32["capt_ratio"] = d_32["capt"]/d_32["industry_capt"]
d_32["ratio_ret"] = d_32["capt_ratio"]*d_32["stkcd_ret"]
d_32 = d_32.groupby(["date"])["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret": "industry_ret"})
d_32 = data[["date", "symbol", "stkcd_ret"]].merge(d_32, right_on = "date", left_on = "date")


# linear model OLS
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params
d_32 = d_32.groupby("symbol").apply(regress, 'stkcd_ret', ['industry_ret']).reset_index().rename(columns = {"industry_ret":"beta"}).merge(d_32, right_on = "symbol", left_on = "symbol")
d_32['abnr_ret'] = d_32["stkcd_ret"] - d_32["intercept"] - d_32["industry_ret"]*d_32["beta"]

d_30['abnr_ret'].corr(d_32['abnr_ret'])

0.9899083518464112

35. 每天有哪些行业的平均收益率超过市场的平均收益率

In [14]:
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1
# 市场收益率
d_30 = data.groupby("date")["capt"].sum().reset_index().rename(columns = {"capt": "mkt_capt"})
d_30 = data.merge(d_30, left_on = "date", right_on = "date")
d_30["capt_ratio"] = d_30["capt"]/d_30["mkt_capt"]
d_30["ratio_ret"] = d_30["capt_ratio"]*d_30["stkcd_ret"]
d_30 = d_30.groupby("date")["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret": "mkt_ret"})

# 行业收益率
d_32 = data.groupby(["date", "industry"])["capt"].sum().reset_index().rename(columns = {"capt": "industry_capt"})
d_32 = data.merge(d_32, left_on = ["date", "industry"], right_on = ["date", "industry"])
d_32["capt_ratio"] = d_32["capt"]/d_32["industry_capt"]
d_32["ratio_ret"] = d_32["capt_ratio"]*d_32["stkcd_ret"]
d_32 = d_32.groupby(["industry", "date"])["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret": "industry_ret"})

d_35 = d_32.merge(d_30, right_on = "date", left_on = "date")
d_35[d_35.industry_ret > d_35.mkt_ret]

,industry,date,industry_ret,mkt_ret
0,AERODEF,20120104,-0.008403,-0.015816
2,AUTO,20120104,-0.013058,-0.015816
3,BANKS,20120104,-0.006737,-0.015816
13,ENERGY,20120104,-0.003714,-0.015816
17,HOUSEDUR,20120104,-0.011340,-0.015816
...,...,...,...,...
3728,HEALTH,20120629,0.018305,0.013166
3730,INDCONG,20120629,0.020436,0.013166
3735,MEDIA,20120629,0.017133,0.013166
3736,MTLMIN,20120629,0.014446,0.013166


36. 每天每个行业对市场的超额收益率是多少？

In [23]:
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1
# 市场收益率
d_30 = data.groupby("date")["capt"].sum().reset_index().rename(columns = {"capt": "mkt_capt"})
d_30 = data.merge(d_30, left_on = "date", right_on = "date")
d_30["capt_ratio"] = d_30["capt"]/d_30["mkt_capt"]
d_30["ratio_ret"] = d_30["capt_ratio"]*d_30["stkcd_ret"]
d_30 = d_30.groupby("date")["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret": "mkt_ret"})

# 行业收益率
d_32 = data.groupby(["date", "industry"])["capt"].sum().reset_index().rename(columns = {"capt": "industry_capt"})
d_32 = data.merge(d_32, left_on = ["date", "industry"], right_on = ["date", "industry"])
d_32["capt_ratio"] = d_32["capt"]/d_32["industry_capt"]
d_32["ratio_ret"] = d_32["capt_ratio"]*d_32["stkcd_ret"]
d_32 = d_32.groupby(["industry", "date"])["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret": "industry_ret"})

d_36 = d_32.merge(d_30, right_on = "date", left_on = "date")

# linear model OLS
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params
d_36 = d_36.merge(d_36.groupby("industry").apply(regress, "industry_ret", ["mkt_ret"]).reset_index().rename(columns = {"mkt_ret" : 'beta'}), right_on = ['industry'], left_on = ['industry'])
d_36["abnr_ret"] = d_36["industry_ret"] - d_36["mkt_ret"]*d_36["beta"] - d_36["intercept"]
d_36[['date', 'industry', 'abnr_ret']]

,date,industry,abnr_ret
0,20120104,AERODEF,0.010403
1,20120105,AERODEF,-0.008207
2,20120106,AERODEF,-0.027298
3,20120109,AERODEF,-0.004859
4,20120110,AERODEF,0.006747
...,...,...,...
3739,20120625,UTILITIE,0.013292
3740,20120626,UTILITIE,-0.008601
3741,20120627,UTILITIE,0.001710
3742,20120628,UTILITIE,0.002332


37. 每天每个行业对去除本行业后的市场超额收益是多少？

In [18]:
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1

# 行业收益率
d_37 = data.groupby(["date", "industry"])["capt"].sum().reset_index().rename(columns = {"capt": "ind_capt"})
d_37 = data.merge(d_37, left_on = ["date", "industry"], right_on = ["date", "industry"])
d_37["capt_ratio"] = d_37["capt"]/d_37["ind_capt"]
d_37["ratio_ret"] = d_37["capt_ratio"] * d_37["stkcd_ret"]
d_37 = d_37.groupby(["date", "industry"])["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret" : "ind_ret"}).merge(d_37.groupby(["date", "industry"])["ind_capt"].first(), right_on = ["industry", "date"], left_on = ['industry', "date"])

# 去除本行业的市场收益率
d_37_group = d_37.groupby(["date"])
d_37_list = []
for name, group in d_37_group:
    for i in range(len(group)):
        a = group.drop(group.index[i])["ind_capt"]/(group.drop(group.index[i])["ind_capt"].sum()).tolist()
        b = group.drop(group.index[i])["ind_ret"].tolist()
        # print(b[:5])
        d_37_list.append(sum(a*b))
d_37["mkt_ret"] = d_37_list

# OLS回归
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params
d_37 = d_37.merge(d_37.groupby("industry").apply(regress, "ind_ret", ["mkt_ret"]).reset_index().rename(columns = {"mkt_ret" : 'beta'}), right_on = ['industry'], left_on = ['industry'])
d_37["abnr_ret"] = d_37["ind_ret"] - d_37["mkt_ret"]*d_37["beta"] - d_37["intercept"]
d_37[['date', 'industry', 'abnr_ret']]

C:\Users\rayst\AppData\Local\Temp\ipykernel_2660\4268839576.py:13: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in d_37_group:


,date,industry,abnr_ret
0,20120104,AERODEF,0.010400
1,20120105,AERODEF,-0.008307
2,20120106,AERODEF,-0.027422
3,20120109,AERODEF,-0.004773
4,20120110,AERODEF,0.006892
...,...,...,...
3739,20120625,UTILITIE,0.013526
3740,20120626,UTILITIE,-0.008831
3741,20120627,UTILITIE,0.001740
3742,20120628,UTILITIE,0.002334


38. 每天分别有多少股票是最近连续3个交易日上涨、下跌的？

In [30]:
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1
data["tag"] = np.where(data["stkcd_ret"] > 0, 1, -1)
d_38 = data.groupby("symbol")["tag"].rolling(3).sum().reset_index()[["symbol", "tag"]].merge(data.groupby(["symbol", "date"])["stkcd_ret"].first().reset_index(), right_index = True, left_index = True)
d_38["up3day"] = np.where(d_38.tag == 3, 1, 0)
d_38["down3day"] = np.where(d_38.tag == -3, 1, 0)
d_38.groupby(["date"])["up3day"].sum().reset_index().merge(
d_38.groupby(["date"])["down3day"].sum().reset_index(), left_on = "date", right_on = "date")

,date,up3day,down3day
0,20120104,0,0
1,20120105,0,0
2,20120106,16,747
3,20120109,71,98
4,20120110,1401,74
...,...,...,...
112,20120625,26,1110
113,20120626,30,943
114,20120627,48,791
115,20120628,90,723


39. 每天分别有多少股票是最近连续3个交易日收益率超过当天市场平均收益率？

In [14]:
data['stkcd_ret'] = data["close"]/data["pre_close"] - 1

# 市场收益率
d_39 = data.groupby("date")["capt"].sum().reset_index().rename(columns = {"capt": "mkt_capt"})
d_39 = data.merge(d_39, left_on = "date", right_on = "date")
d_39["capt_ratio"] = d_39["capt"]/d_39["mkt_capt"]
d_39["ratio_ret"] = d_39["capt_ratio"]*d_39["stkcd_ret"]
d_39 = d_39.groupby("date")["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret": "mkt_ret"})

# 合并两个收益率
d_39 = data[["date", "symbol", "stkcd_ret"]].merge(d_39, right_on = "date", left_on = "date").sort_values(by = ["symbol", "date"])

# 做标记
d_39["tag"] = np.where(d_39.stkcd_ret > d_39.mkt_ret, 1, 0)
d_39 = d_39.groupby(["symbol"])["tag"].rolling(3).sum().reset_index()[["symbol", "tag"]].merge(
    data.sort_values(by = ["symbol", "date"]).groupby(["symbol", "date"])["stkcd_ret"].first().reset_index(),
    right_index = True, 
    left_index = True
)
d_39["above_mkt"] = np.where(d_39.tag == 3, 1, 0)
d_39.groupby(["date"])["above_mkt"].sum().reset_index()

,date,above_mkt
0,20120104,0
1,20120105,0
2,20120106,81
3,20120109,95
4,20120110,785
...,...,...
112,20120625,251
113,20120626,258
114,20120627,242
115,20120628,275


40. 每天分别有多少股票是最新5个交易日中至少有4个交易日的收益率超过当天市场平均收益率？

In [20]:
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1

# 市场收益率
d_40 = data.groupby("date")["capt"].sum().reset_index().rename(columns = {"capt": "mkt_capt"})
d_40 = data.merge(d_40, left_on = "date", right_on = "date")
d_40["capt_ratio"] = d_40["capt"]/d_40["mkt_capt"]
d_40["ratio_ret"] = d_40["capt_ratio"]*d_40["stkcd_ret"]
d_40 = d_40.groupby("date")["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret": "mkt_ret"})

# 合并两个收益率
d_40 = data[["date", "symbol", "stkcd_ret"]].merge(d_40, right_on = "date", left_on = "date").sort_values(by = ["symbol", "date"])

# 做标记
d_40["tag"] = np.where(d_40.stkcd_ret > d_40.mkt_ret, 1, 0)
d_40 = d_40.groupby(["symbol"])["tag"].rolling(5).sum().reset_index()[["symbol", "tag"]].merge(
    data.sort_values(by = ["symbol", "date"]).groupby(["symbol", "date"])["stkcd_ret"].first().reset_index(),
    right_index = True,
    left_index = True
)

# 出结果
d_40["5day_4above"] = np.where(d_40.tag >= 4, 1, 0)
d_40.groupby(["date"])["5day_4above"].sum().reset_index()

,date,5day_4above
0,20120104,0
1,20120105,0
2,20120106,0
3,20120109,0
4,20120110,235
...,...,...
112,20120625,405
113,20120626,338
114,20120627,365
115,20120628,375


41. 每个月中，个股月收益超过市场月收益1倍以上的股票有哪些？

In [38]:
data["date"] = data["date"].astype(str)
data["year"] = data["date"].str.slice(stop = 4)
data["month"] = data["date"].str.slice(start = 4, stop = 6)

# 股票月收益率
d_41_1 = data.groupby(["symbol", "year", "month"])["close"].first().reset_index().rename(columns = {"close" : "month_start"}).merge(
    data.groupby(["symbol", "year", "month"])["close"].last().reset_index().rename(columns = {"close" : "month_end"}), 
    right_on = ["symbol", "year", "month"], 
    left_on = ["symbol", "year", "month"])
d_41_1["stkcd_ret_monthly"] = d_41_1["month_start"]/d_41_1["month_end"] - 1

# 市场月收益率
d_41_2 = data.groupby("date")["capt"].sum().reset_index().rename(columns = {"capt": "mkt_capt"})
d_41_2 = data.merge(d_41_2, left_on = "date", right_on = "date")
d_41_2["capt_ratio"] = d_41_2["capt"]/d_41_2["mkt_capt"]
d_41_2["ratio_close"] = d_41_2["capt_ratio"]*d_41_2["close"]
d_41_2 = d_41_2.groupby(["date", "year", "month"])["ratio_close"].sum().reset_index().rename(columns = {"ratio_close": "mkt_close"})
d_41_2 = d_41_2.groupby(["year", "month"])["mkt_close"].first().reset_index().rename(columns = {"mkt_close" : "mkt_month_start"}).merge(
    d_41_2.groupby(["year", "month"])["mkt_close"].last().reset_index().rename(columns = {"mkt_close":"mkt_month_end"}),
    right_on = ["year", "month"],
    left_on = ["year", "month"]
    )
d_41_2["mkt_ret_monthly"] = d_41_2["mkt_month_end"]/d_41_2["mkt_month_start"] - 1

# 合并数据集
d_41 = d_41_1.merge(d_41_2, how = "left", left_on = ["year", "month"], right_on = ["year", "month"])[["year", "month", "symbol", "stkcd_ret_monthly", "mkt_ret_monthly"]]

d_41["tag"] = np.where(d_41.stkcd_ret_monthly/d_41.mkt_ret_monthly >= 2, 1, 0)

d_41[d_41.tag == 1].sort_values(by = ["year", "month"]).reset_index()[["year", "month", "symbol"]]

,year,month,symbol
0,2012,01,000026.SZ
1,2012,01,000028.SZ
2,2012,01,000062.SZ
3,2012,01,000063.SZ
4,2012,01,000151.SZ
...,...,...,...
1857,2012,06,600998.SH
1858,2012,06,601318.SH
1859,2012,06,601566.SH
1860,2012,06,601991.SH


42. 每个月中，个股月收益超过行业月收益1倍以上的股票有哪些？

In [37]:
data["date"] = data["date"].astype(str)
data["year"] = data["date"].str.slice(stop = 4)
data["month"] = data["date"].str.slice(start = 4, stop = 6)

# 股票月收益率
d_42_1 = data.groupby(["symbol", "industry", "year", "month"])["close"].first().reset_index().rename(columns = {"close" : "month_start"}).merge(
    data.groupby(["symbol", "industry", "year", "month"])["close"].last().reset_index().rename(columns = {"close" : "month_end"}), 
    right_on = ["symbol", "industry", "year", "month"], 
    left_on = ["symbol", "industry", "year", "month"])
d_42_1["stkcd_ret_monthly"] = d_42_1["month_start"]/d_42_1["month_end"] - 1

# 行业月收益率
d_42_2 = data.groupby(["date", "industry"])["capt"].sum().reset_index().rename(columns = {"capt": "industry_capt"})
d_42_2 = data.merge(d_42_2, left_on = ["date", "industry"], right_on = ["date", "industry"])
d_42_2["capt_ratio"] = d_42_2["capt"]/d_42_2["industry_capt"]
d_42_2["ratio_close"] = d_42_2["capt_ratio"]*d_42_2["close"]
d_42_2 = d_42_2.groupby(["date", "year", "month", "industry"])["ratio_close"].sum().reset_index().rename(columns = {"ratio_close": "ind_close"})
d_42_2 = d_42_2.groupby(["year", "month", "industry"])["ind_close"].first().reset_index().rename(columns = {"ind_close" : "ind_month_start"}).merge(
    d_42_2.groupby(["year", "month", "industry"])["ind_close"].last().reset_index().rename(columns = {"ind_close":"ind_month_end"}),
    right_on = ["year", "month", "industry"],
    left_on = ["year", "month", "industry"]
    )
d_42_2["ind_ret_monthly"] = d_42_2["ind_month_end"]/d_42_2["ind_month_start"] - 1

# 合并数据集
d_42 = d_42_1.merge(d_42_2, how = "left", left_on = ["year", "month", "industry"], right_on = ["year", "month", "industry"])[["year", "month", "symbol", "stkcd_ret_monthly", "ind_ret_monthly"]]

d_42["tag"] = np.where(d_42.stkcd_ret_monthly/d_42.ind_ret_monthly >= 2, 1, 0)

d_42[d_42.tag == 1].sort_values(by = ["year", "month"]).reset_index()[["year", "month", "symbol"]]

,year,month,symbol
0,2012,01,000019.SZ
1,2012,01,000033.SZ
2,2012,01,000034.SZ
3,2012,01,000065.SZ
4,2012,01,000069.SZ
...,...,...,...
1643,2012,06,601599.SH
1644,2012,06,601688.SH
1645,2012,06,601788.SH
1646,2012,06,601991.SH


43. 每个股票的收益率对市场收益率的相关系数最高的10个股票是哪些？

In [33]:
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1

# 市场收益率
d_43 = data.groupby("date")["capt"].sum().reset_index().rename(columns = {"capt": "mkt_capt"})
d_43 = data.merge(d_43, left_on = "date", right_on = "date")
d_43["capt_ratio"] = d_43["capt"]/d_43["mkt_capt"]
d_43["ratio_ret"] = d_43["capt_ratio"]*d_43["stkcd_ret"]
d_43 = d_43.groupby("date")["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret": "mkt_ret"})

# 与股票收益率合并
d_43 = data[["date", "symbol", "stkcd_ret"]].merge(d_43, how = "left", left_on = "date", right_on = "date")

# 求相关系数
d_43 = d_43.groupby("symbol")[["stkcd_ret", "mkt_ret"]].corr().reset_index()
d_43.groupby("symbol")["stkcd_ret"].last().reset_index().rename(columns = {"stkcd_ret" : "correlation"}).sort_values(by = ["correlation"], ascending = False).head(10)

,symbol,correlation
1484,300331.SZ,1.000000
1902,600508.SH,0.927955
2310,601101.SH,0.909776
245,000685.SZ,0.897915
2370,601666.SH,0.893953
553,002082.SZ,0.893250
683,002212.SZ,0.891920
2292,601001.SH,0.891893
59,000089.SZ,0.891447
2323,601168.SH,0.888806


44. 每个行业日收益率的历史波动率是多少？

In [39]:
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1

# 行业收益率
d_44 = data.groupby(["date", "industry"])["capt"].sum().reset_index().rename(columns = {"capt": "ind_capt"})
d_44 = data.merge(d_44, left_on = ["date", "industry"], right_on = ["date", "industry"])
d_44["capt_ratio"] = d_44["capt"]/d_44["ind_capt"]
d_44["ratio_ret"] = d_44["capt_ratio"] * d_44["stkcd_ret"]
d_44 = d_44.groupby(["date", "industry"])["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret" : "ind_ret"}).merge(d_44.groupby(["date", "industry"])["ind_capt"].first(), right_on = ["industry", "date"], left_on = ['industry', "date"])

d_44.groupby(["industry"])["ind_ret"].std().reset_index().rename(columns = {"ind_ret":"industry_standard_deviation"})

,industry,industry_standard_deviation
0,AERODEF,0.017500
1,AIRLINE,0.017127
2,AUTO,0.015097
3,BANKS,0.008106
4,BEV,0.016408
5,BLDPROD,0.016608
6,CHEM,0.017266
7,CNSTENG,0.015310
8,COMSERV,0.017257
9,CONMAT,0.021500


45. 各个行业的日收益率的相关系数矩阵如何？哪两个行业相关性最高、最低？

In [2]:
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1

# 行业收益率
d_45 = data.groupby(["date", "industry"])["capt"].sum().reset_index().rename(columns = {"capt": "ind_capt"})
d_45 = data.merge(d_45, left_on = ["date", "industry"], right_on = ["date", "industry"])
d_45["capt_ratio"] = d_45["capt"]/d_45["ind_capt"]
d_45["ratio_ret"] = d_45["capt_ratio"] * d_45["stkcd_ret"]
d_45 = d_45.groupby(["date", "industry"])["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret" : "ind_ret"}).merge(d_45.groupby(["date", "industry"])["ind_capt"].first(), right_on = ["industry", "date"], left_on = ['industry', "date"])[["date", "industry", "ind_ret"]]

d_45 = d_45.pivot_table(index = ["date"],
                columns = "industry",
                values = "ind_ret")
d_45.corr()

industry,AERODEF,AIRLINE,AUTO,BANKS,BEV,BLDPROD,CHEM,CNSTENG,COMSERV,CONMAT,...,MATERIAL,MEDIA,MTLMIN,PERSPRD,RDRLTRAN,REALEST,RETAIL,SOFTWARE,TRDDIST,UTILITIE
industry,,,,,,,,,,,,,,,,,,,,,
AERODEF,1.000000,0.715879,0.800998,0.465783,0.471870,0.764393,0.800025,0.792307,0.801909,0.739309,...,0.758858,0.729190,0.720849,0.770809,0.696612,0.691247,0.764706,0.783704,0.786471,0.681218
AIRLINE,0.715879,1.000000,0.818696,0.654099,0.540875,0.721121,0.811375,0.829383,0.723600,0.767873,...,0.733148,0.710533,0.796893,0.732164,0.850983,0.734250,0.767441,0.733805,0.804745,0.775823
AUTO,0.800998,0.818696,1.000000,0.606801,0.533968,0.853847,0.879147,0.897974,0.849278,0.838886,...,0.802517,0.811508,0.823657,0.837697,0.874041,0.828531,0.862460,0.822306,0.878464,0.825702
BANKS,0.465783,0.654099,0.606801,1.000000,0.383583,0.492436,0.560457,0.627839,0.493130,0.572595,...,0.481197,0.519133,0.626276,0.529474,0.718676,0.585352,0.559214,0.478655,0.630844,0.608247
BEV,0.471870,0.540875,0.533968,0.383583,1.000000,0.584932,0.631810,0.590255,0.605060,0.525400,...,0.573882,0.622516,0.548910,0.647726,0.493790,0.490988,0.655867,0.613208,0.589354,0.596221
BLDPROD,0.764393,0.721121,0.853847,0.492436,0.584932,1.000000,0.924797,0.876360,0.908562,0.819237,...,0.865592,0.853001,0.803298,0.892116,0.775380,0.794407,0.876892,0.884670,0.881484,0.797645
CHEM,0.800025,0.811375,0.879147,0.560457,0.631810,0.924797,1.000000,0.903770,0.900213,0.851568,...,0.894377,0.886290,0.907845,0.899530,0.842615,0.785785,0.903980,0.907963,0.943814,0.848540
CNSTENG,0.792307,0.829383,0.897974,0.627839,0.590255,0.876360,0.903770,1.000000,0.868062,0.916646,...,0.833593,0.815147,0.864339,0.849566,0.862651,0.855420,0.873918,0.829065,0.912860,0.835865
COMSERV,0.801909,0.723600,0.849278,0.493130,0.605060,0.908562,0.900213,0.868062,1.000000,0.788934,...,0.867183,0.861177,0.762735,0.893613,0.771725,0.784265,0.889649,0.915850,0.867255,0.826870


In [43]:
d_45_coef = d_45.corr()
d_45_coef.stack()[d_45_coef.stack() != 1].index[np.argmax(d_45_coef.values[d_45_coef.values != 1])]

('ELECEQP', 'HDWRSEMI')

In [36]:
d_45_coef.stack().index[np.argmin(d_45_coef.values)]

('BANKS', 'BEV')

46. 各个行业的收益率对市场收益率的相关系数由高到低排列如何？

In [47]:
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1

# 行业收益率
d_46 = data.groupby(["date", "industry"])["capt"].sum().reset_index().rename(columns = {"capt": "ind_capt"})
d_46 = data.merge(d_46, left_on = ["date", "industry"], right_on = ["date", "industry"])
d_46["capt_ratio"] = d_46["capt"]/d_46["ind_capt"]
d_46["ratio_ret"] = d_46["capt_ratio"] * d_46["stkcd_ret"]
d_46 = d_46.groupby(["date", "industry"])["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret" : "ind_ret"}).merge(d_46.groupby(["date", "industry"])["ind_capt"].first(), right_on = ["industry", "date"], left_on = ['industry', "date"])

d_46.groupby(["industry"])["ind_ret"].std().reset_index().rename(columns = {"ind_ret":"industry_standard_deviation"}).sort_values(by = ["industry_standard_deviation"], ascending = False)

,industry,industry_standard_deviation
9,CONMAT,0.021500
18,INDCONG,0.019694
11,DVFININS,0.018947
21,MARINE,0.018242
29,SOFTWARE,0.018210
23,MEDIA,0.018032
0,AERODEF,0.017500
24,MTLMIN,0.017472
27,REALEST,0.017361
6,CHEM,0.017266


47. 每个月总成交额比上个月下降幅度最大的行业是哪个？

In [65]:
data["date"] = data["date"].astype(str)
data["year"] = data["date"].str.slice(stop = 4)
data["month"] = data["date"].str.slice(start = 4, stop = 6)

d_47 = data.groupby(["industry", "year", "month"])["amount"].sum().reset_index()
d_47["amount_lag"] = d_47.groupby(["industry"])["amount"].shift(1)
d_47["amount_change"] = d_47["amount"] - d_47["amount_lag"] 
d_47 = d_47.groupby(["year", "month"])["amount_change"].min().reset_index().rename(columns = {"amount_change" : "amount_change_min"}).merge(d_47, how = "right", right_on = ["year", "month"], left_on = ["year", "month"])
d_47[d_47.amount_change_min == d_47.amount_change][["year", "month", "industry", "amount_change"]].sort_values(by = ["year", "month"]).reset_index()

,index,year,month,industry,amount_change
0,127,2012,02,MARINE,4.090901e+09
1,56,2012,03,CONMAT,-2.175255e+10
2,147,2012,04,MTLMIN,-1.657756e+11
3,166,2012,05,REALEST,-4.528376e+09
4,149,2012,06,MTLMIN,-1.232720e+11


48. 数据当中各个股票的最大回撤幅度是多少？（最大回撤是从一个高点到低点的降幅的最大值）

In [21]:
d_48 = data.sort_values(by = ["symbol", "date"])
d_48_group = d_48.groupby('symbol')

d_48_list = []
for name, group in d_48_group:
    for i in range(len(group)):
        # a = group.iloc[i,:]["high"]
        # b = group.iloc[i:(len(group) + 1), :]["low"]
        # c = a - b
        d_48_list.append(max(group.iloc[i,:]["high"] - group.iloc[i:(len(group) + 1), :]["low"]))
d_48["max_retreat"] = d_48_list
d_48.groupby(["symbol"])["max_retreat"].max().reset_index()

,symbol,max_retreat
0,000001.SZ,3.33
1,000002.SZ,0.92
2,000004.SZ,2.12
3,000005.SZ,1.74
4,000006.SZ,1.47
...,...,...
2420,603002.SH,2.27
2421,603123.SH,4.53
2422,603128.SH,1.78
2423,603333.SH,2.99


48. 每只股票的胜率是多少？（胜率是每天收益率为正数的概率）

In [29]:
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1
data["tag"] = np.where(data.stkcd_ret > 0, 1, 0)
d_48 = data.groupby(["symbol"])["tag"].sum().reset_index().merge(
    data.groupby(["symbol"])["date"].count().reset_index(),
    right_on = "symbol",
    left_on = "symbol"
)
d_48["gain_ratio"] = d_48["tag"]/d_48["date"]
d_48[["symbol", "gain_ratio"]]

,symbol,gain_ratio
0,000001.SZ,0.418803
1,000002.SZ,0.470085
2,000004.SZ,0.512821
3,000005.SZ,0.153846
4,000006.SZ,0.512821
...,...,...
2420,603002.SH,0.433333
2421,603123.SH,0.317073
2422,603128.SH,0.478261
2423,603333.SH,0.358974


50. 每只股票的盈亏比是多少？（盈亏比是正收益之和与负收益之和的比值的绝对值）

In [36]:
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1

d_50 = data[data.stkcd_ret > 0].groupby(["symbol"])["stkcd_ret"].sum().reset_index().rename(columns = {"stkcd_ret": "gain"}).merge(
    data[data.stkcd_ret < 0].groupby(["symbol"])["stkcd_ret"].sum().abs().reset_index().rename(columns = {"stkcd_ret": "loss"}),
    right_on = "symbol",
    left_on = "symbol"
)
d_50["gl_ratio"] = d_50["gain"]/d_50["loss"]
d_50[["symbol", "gl_ratio"]]

,symbol,gl_ratio
0,000001.SZ,0.971861
1,000002.SZ,1.279686
2,000004.SZ,1.082869
3,000005.SZ,0.553243
4,000006.SZ,1.371993
...,...,...
2388,603002.SH,3.327521
2389,603123.SH,1.415443
2390,603128.SH,1.266408
2391,603333.SH,1.020868


51. 市场的胜率是多少？（市场收益率为正的概率）

In [43]:
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1

# 市场收益率
d_51 = data.groupby("date")["capt"].sum().reset_index().rename(columns = {"capt": "mkt_capt"})
d_51 = data.merge(d_51, left_on = "date", right_on = "date")
d_51["capt_ratio"] = d_51["capt"]/d_51["mkt_capt"]
d_51["ratio_ret"] = d_51["capt_ratio"]*d_51["stkcd_ret"]
d_51 = d_51.groupby("date")["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret": "mkt_ret"})

d_51[d_51.mkt_ret > 0]["date"].count()/d_51["date"].count()


0.48717948717948717

52. 市场的盈亏比是多少？（市场中每个股票的市值加权正收益和市值加权负收益比）

In [44]:
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1

# 市场收益率
d_52 = data.groupby("date")["capt"].sum().reset_index().rename(columns = {"capt": "mkt_capt"})
d_52 = data.merge(d_52, left_on = "date", right_on = "date")
d_52["capt_ratio"] = d_52["capt"]/d_52["mkt_capt"]
d_52["ratio_ret"] = d_52["capt_ratio"]*d_52["stkcd_ret"]
d_52 = d_52.groupby("date")["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret": "mkt_ret"})

d_52[d_52.mkt_ret > 0]["mkt_ret"].sum()/d_52[d_52.mkt_ret < 0]["mkt_ret"].abs().sum()

1.1473732397390033

53. 每个行业的胜率是多少？

In [51]:
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1

# 行业收益率
d_53 = data.groupby(["date", "industry"])["capt"].sum().reset_index().rename(columns = {"capt": "ind_capt"})
d_53 = data.merge(d_53, left_on = ["date", "industry"], right_on = ["date", "industry"])
d_53["capt_ratio"] = d_53["capt"]/d_53["ind_capt"]
d_53["ratio_ret"] = d_53["capt_ratio"] * d_53["stkcd_ret"]
d_53 = d_53.groupby(["date", "industry"])["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret" : "ind_ret"}).merge(d_53.groupby(["date", "industry"])["ind_capt"].first(), right_on = ["industry", "date"], left_on = ['industry', "date"])

d_53 = d_53[d_53.ind_ret > 0].groupby(["industry"])["date"].count().reset_index().rename(columns = {"date": "gain_amount"}).merge(
    d_53.groupby(["industry"])["date"].count().reset_index().rename(columns = {"date": "amount"}),
    right_on = "industry",
    left_on = "industry"
)

d_53["gain_ratio"] = d_53["gain_amount"]/d_53["amount"]
d_53[["industry", "gain_ratio"]]

,industry,gain_ratio
0,AERODEF,0.521368
1,AIRLINE,0.478632
2,AUTO,0.470085
3,BANKS,0.461538
4,BEV,0.547009
5,BLDPROD,0.538462
6,CHEM,0.495726
7,CNSTENG,0.547009
8,COMSERV,0.521368
9,CONMAT,0.529915


54. 每个行业的盈亏比是多少？（行业盈亏比是行业内每个股票的市值加权的正收益率和市值加权的负收益率之比）

In [52]:
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1

# 行业收益率
d_54 = data.groupby(["date", "industry"])["capt"].sum().reset_index().rename(columns = {"capt": "ind_capt"})
d_54 = data.merge(d_54, left_on = ["date", "industry"], right_on = ["date", "industry"])
d_54["capt_ratio"] = d_54["capt"]/d_54["ind_capt"]
d_54["ratio_ret"] = d_54["capt_ratio"] * d_54["stkcd_ret"]
d_54 = d_54.groupby(["date", "industry"])["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret" : "ind_ret"}).merge(d_54.groupby(["date", "industry"])["ind_capt"].first(), right_on = ["industry", "date"], left_on = ['industry', "date"])

d_54 = d_54[d_54.ind_ret > 0].groupby(["industry"])["ind_ret"].sum().reset_index().rename(columns = {"ind_ret": "gain"}).merge(
    d_54[d_54.ind_ret < 0].groupby(["industry"])["ind_ret"].sum().abs().reset_index().rename(columns = {"ind_ret": "loss"}),
    right_on = "industry",
    left_on = "industry"
)

d_54["gl_ratio"] = d_54["gain"]/d_54["loss"]
d_54[["industry", "gl_ratio"]]

,industry,gl_ratio
0,AERODEF,0.997362
1,AIRLINE,1.071538
2,AUTO,1.120609
3,BANKS,1.015873
4,BEV,1.281267
5,BLDPROD,1.269879
6,CHEM,1.114915
7,CNSTENG,1.241538
8,COMSERV,1.199034
9,CONMAT,1.027472


55. 是否存在股票的月成交额超过所在行业当月中某一天总成交额的情况？

In [9]:
data["date"] = data["date"].astype(str)
data["year"] = data["date"].str.slice(stop = 4)
data["month"] = data["date"].str.slice(start = 4, stop = 6)

# 股票的月成交额
d_55 = data.groupby(["year", "month", "industry", "symbol"])["amount"].sum().reset_index().rename(columns = {"amount" : "stkcd_amount"}).merge(
    # 行业的日交易额
    data.groupby(["date", "year", "month", "industry"])["amount"].sum().reset_index().rename(columns = {"amount" : "ind_amount"}),
    how = "inner",
    left_on = ["year", "month", "industry"],
    right_on = ["year", "month", "industry"]
)

d_55[d_55.stkcd_amount > d_55.ind_amount][["year", "month", "industry", "symbol", "stkcd_amount", "ind_amount"]]


,year,month,industry,symbol,stkcd_amount,ind_amount
0,2012,01,AERODEF,000768.SZ,1.460111e+09,4.933312e+08
1,2012,01,AERODEF,000768.SZ,1.460111e+09,5.488054e+08
2,2012,01,AERODEF,000768.SZ,1.460111e+09,6.273671e+08
3,2012,01,AERODEF,000768.SZ,1.460111e+09,5.059227e+08
4,2012,01,AERODEF,000768.SZ,1.460111e+09,8.318035e+08
...,...,...,...,...,...,...
278024,2012,06,UTILITIE,600795.SH,3.541810e+09,3.527167e+09
278025,2012,06,UTILITIE,600795.SH,3.541810e+09,2.805190e+09
278026,2012,06,UTILITIE,600795.SH,3.541810e+09,2.673393e+09
278027,2012,06,UTILITIE,600795.SH,3.541810e+09,2.899220e+09


56. 每天每个行业编入、编出的股票各有多少？

In [4]:
d_56 = data.groupby(["date", "industry"]).agg({'symbol': lambda x: list(x)}).reset_index().sort_values(by = ["industry", "date"])
d_56_group = d_56.groupby(["industry"])
d_56_list1 = []
d_56_list2 = []
for name, group in d_56_group:
    for i in range(0, len(group)):
        if i == 0:
            d_56_list1.append(0)
            d_56_list2.append(0)
        else:
            a = set(group.iloc[i, :]["symbol"])
            b = set(group.iloc[(i-1), :]["symbol"])
            c = a & b
            a1 = len(a - c)
            b1 = len(b - c)
            d_56_list1.append(a1)
            d_56_list2.append(b1)
d_56["in_symbol"] = d_56_list1
d_56["out_symbol"] = d_56_list2
d_56[["date", "industry", "in_symbol", "out_symbol"]]


,date,industry,in_symbol,out_symbol
0,20120104,AERODEF,0,0
32,20120105,AERODEF,0,0
64,20120106,AERODEF,0,0
96,20120109,AERODEF,0,0
128,20120110,AERODEF,0,0
...,...,...,...,...
3615,20120625,UTILITIE,0,0
3647,20120626,UTILITIE,0,0
3679,20120627,UTILITIE,0,0
3711,20120628,UTILITIE,0,0


57. 每天每个行业内股票收益率的标准差是多少？

In [4]:
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1
data.groupby(["date", "industry"])["stkcd_ret"].std().reset_index().rename(columns = {"stkcd_ret": "stkcd_ret_std"})

,date,industry,stkcd_ret_std
0,20120104,AERODEF,0.028369
1,20120104,AIRLINE,0.011949
2,20120104,AUTO,0.024044
3,20120104,BANKS,0.008446
4,20120104,BEV,0.025690
...,...,...,...
3739,20120629,REALEST,0.018059
3740,20120629,RETAIL,0.015018
3741,20120629,SOFTWARE,0.023618
3742,20120629,TRDDIST,0.024542


58. 每天每个行业内股票收益率的标准差的相关性如何？

In [6]:
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1
d_58 = data.groupby(["date", "industry"])["stkcd_ret"].std().reset_index().rename(columns = {"stkcd_ret": "stkcd_ret_std"})
d_58 = d_58.pivot_table(index = ["date"],
                columns = "industry",
                values = "stkcd_ret_std")
d_58.corr()

industry,AERODEF,AIRLINE,AUTO,BANKS,BEV,BLDPROD,CHEM,CNSTENG,COMSERV,CONMAT,...,MATERIAL,MEDIA,MTLMIN,PERSPRD,RDRLTRAN,REALEST,RETAIL,SOFTWARE,TRDDIST,UTILITIE
industry,,,,,,,,,,,,,,,,,,,,,
AERODEF,1.000000,0.115230,0.331956,0.080215,0.024979,0.235179,0.188611,0.216828,0.257316,0.390574,...,-0.039475,0.055022,0.237747,0.073751,0.234620,0.240815,0.197279,0.461433,0.128383,0.257165
AIRLINE,0.115230,1.000000,0.222355,0.024462,0.022503,0.119220,0.167030,0.301270,0.125237,0.170596,...,-0.065845,0.004517,0.153300,0.207793,0.238502,0.234945,0.275787,0.009784,0.320399,0.132329
AUTO,0.331956,0.222355,1.000000,0.072840,0.263742,0.520673,0.369540,0.324969,0.574584,0.289746,...,0.120343,0.101366,0.346575,0.402236,0.460378,0.459369,0.428962,0.208919,0.363675,0.322791
BANKS,0.080215,0.024462,0.072840,1.000000,0.060737,0.160753,0.157011,-0.010451,0.017724,0.203366,...,-0.038268,-0.038437,0.137094,0.149524,0.132156,0.192628,0.116732,-0.042062,0.072882,0.263463
BEV,0.024979,0.022503,0.263742,0.060737,1.000000,0.352289,0.114483,0.211685,0.139490,0.118357,...,0.101116,0.154699,0.367896,0.198618,0.053848,0.079432,0.205947,0.122781,0.162351,0.173413
BLDPROD,0.235179,0.119220,0.520673,0.160753,0.352289,1.000000,0.414073,0.313940,0.365990,0.210249,...,0.044679,0.002654,0.384144,0.277338,0.405853,0.321022,0.275536,0.222124,0.272960,0.349810
CHEM,0.188611,0.167030,0.369540,0.157011,0.114483,0.414073,1.000000,0.161459,0.128857,0.146228,...,0.025712,-0.002682,0.338836,0.189411,0.542983,0.364189,0.309889,0.149510,0.421167,0.236075
CNSTENG,0.216828,0.301270,0.324969,-0.010451,0.211685,0.313940,0.161459,1.000000,0.201579,0.234814,...,0.008943,0.020133,0.218982,0.243457,0.255077,0.214717,0.292674,0.169888,0.279617,0.239629
COMSERV,0.257316,0.125237,0.574584,0.017724,0.139490,0.365990,0.128857,0.201579,1.000000,0.288058,...,-0.033835,0.064871,0.320102,0.293915,0.319976,0.248270,0.164136,0.169654,0.212449,0.320810


59. 每天计算出成交额的z-score（减去均值除以标准差），该指标能解释下一天个股超额收益率的多少比例？

In [48]:
# 1. z-score计算
d_59 = data.groupby(["date"]).agg({"amount":["mean", "std"]}).reset_index()
d_59.columns = ['date', 'amount_mean', "amount_std"]
d_59 = data[["date", "symbol", "amount"]].merge(d_59, right_on = "date", left_on = "date", how = "left")
d_59["z_score"] = (d_59["amount"]-d_59["amount_mean"])/d_59["amount_std"]
d_59 = d_59[["date", "symbol", "z_score"]]

# 2. 每只股票的超额收益率
# 2.1 市场收益率
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1
d_59_ret = data.groupby("date")["capt"].sum().reset_index().rename(columns = {"capt": "mkt_capt"})
d_59_ret = data.merge(d_59_ret, left_on = "date", right_on = "date")
d_59_ret["capt_ratio"] = d_59_ret["capt"]/d_59_ret["mkt_capt"]
d_59_ret["ratio_ret"] = d_59_ret["capt_ratio"]*d_59_ret["stkcd_ret"]
d_59_ret = d_59_ret.groupby("date")["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret": "mkt_ret"})

d_59_ret = data[["date", "symbol", "stkcd_ret"]].merge(d_59_ret, how = "left", right_on = "date", left_on = "date")

# 2.2 循环回归
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

d_59_ret = d_59_ret.merge(d_59_ret.groupby("symbol").apply(regress, "stkcd_ret", ["mkt_ret"]).reset_index().rename(columns = {"mkt_ret" : 'beta'}), right_on = ['symbol'], left_on = ['symbol'])
d_59_ret["abnr_ret"] = d_59_ret["stkcd_ret"] - d_59_ret["mkt_ret"]*d_59_ret["beta"] - d_59_ret["intercept"]
d_59_ret = d_59_ret[['date', 'symbol', 'abnr_ret']]

# 3. 合并做两个回归
d_59 = d_59_ret.merge(d_59, right_on = ["date", "symbol"], left_on = ["date", "symbol"])
d_59["abnr_ret_lag"] = d_59.groupby(["symbol"])["abnr_ret"].shift(-1)
d_59 = d_59[~np.isnan(d_59.abnr_ret_lag)]

def regress_sq(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.rsquared

d_59.groupby("symbol").apply(regress_sq, "z_score", ["abnr_ret_lag"]).reset_index().rename(columns = {0 : "r_squared"})

d:\software\anaconda\Lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
d:\software\anaconda\Lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
d:\software\anaconda\Lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


,symbol,r_squared
0,000001.SZ,0.017435
1,000002.SZ,0.009520
2,000004.SZ,0.000672
3,000005.SZ,0.082448
4,000006.SZ,0.039915
...,...,...
2420,603002.SH,0.057117
2421,603123.SH,0.328365
2422,603128.SH,0.003183
2423,603333.SH,0.096465


60. 每个股票的收益率和300、500指数收益率可以回归出一个截距项和2个beta，这两个beta的分布如何？

In [52]:
#计算300和500指数收益率
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1
data["index_w500_ret"] = data["stkcd_ret"]*data["index_w500"]
data["index_w300_ret"] = data["stkcd_ret"]*data["index_w300"]
d_60_index = data.groupby(["date"]).agg({"index_w500_ret":"sum", "index_w300_ret":"sum"}).reset_index()
d_60 = data[["date", "symbol", "stkcd_ret"]].merge(d_60_index, right_on = "date", left_on = "date", how = "left")

import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

d_60 = d_60.groupby(["symbol"]).apply(regress, "stkcd_ret", ["index_w500_ret", "index_w300_ret"]).reset_index()
d_60.agg({"index_w500_ret": ["mean", "std"], "index_w300_ret": ["mean", "std"]})

,index_w500_ret,index_w300_ret
mean,1.122131,-0.154989
std,0.851576,0.882951


61. 每天开盘后到最高价涨幅最大的100只股票同样也是全天（昨收到今收）涨幅最大的100只股票的比例是多少？

In [34]:
data["open_high"] = data["high"]/data["open"] - 1
data["close_close"] = data["close"]/data["pre_close"] - 1

d_61_oh = data.sort_values(by = ["date", "open_high"]).groupby("date")[["symbol", "date"]].tail(100).reset_index()[["symbol", "date"]].rename(columns = {"symbol": "open_high_symbol"})

d_61_cc = data.sort_values(by = ["date", "close_close"]).groupby("date")[["symbol", "date"]].tail(100).reset_index()[["symbol", "date"]].rename(columns = {"symbol": "close_close_symbol"})

d_61 = pd.merge(d_61_cc, d_61_oh, left_index = True, right_index = True, how = "left").drop(columns = ["date_x"]).rename(columns = {"date_y": "date"})

# 每天设定成一个list
d_61 = d_61.groupby(["date"]).agg({'open_high_symbol': lambda x: list(x), 'close_close_symbol':lambda x: list(x)}).reset_index()

# 计算交集
d_61["intersection_number"] = [len(set(a) & set(b)) for a, b in zip(d_61.open_high_symbol, d_61.close_close_symbol)]

# 计算比例
d_61[d_61.intersection_number > 100]["intersection_number"].count()/d_61["intersection_number"].count()


0.0

62. 每天计算最近三天每天对市场的超额收益率都排进当天前100的股票有哪些？

In [23]:
# 1. 每只股票的超额收益率
# 1.1 市场收益率
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1
d_62_ret = data.groupby("date")["capt"].sum().reset_index().rename(columns = {"capt": "mkt_capt"})
d_62_ret = data.merge(d_62_ret, left_on = "date", right_on = "date")
d_62_ret["capt_ratio"] = d_62_ret["capt"]/d_62_ret["mkt_capt"]
d_62_ret["ratio_ret"] = d_62_ret["capt_ratio"]*d_62_ret["stkcd_ret"]
d_62_ret = d_62_ret.groupby("date")["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret": "mkt_ret"})

d_62_ret = data[["date", "symbol", "stkcd_ret"]].merge(d_62_ret, how = "left", right_on = "date", left_on = "date")

# 1.2 循环回归
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

d_62_ret = d_62_ret.merge(d_62_ret.groupby("symbol").apply(regress, "stkcd_ret", ["mkt_ret"]).reset_index().rename(columns = {"mkt_ret" : 'beta'}), right_on = ['symbol'], left_on = ['symbol'])
d_62_ret["abnr_ret"] = d_62_ret["stkcd_ret"] - d_62_ret["mkt_ret"]*d_62_ret["beta"] - d_62_ret["intercept"]
d_62_ret = d_62_ret[['date', 'symbol', 'abnr_ret']].sort_values(by = ["date", "abnr_ret"], ascending = [True, False])

# 2. 设定tag进行rolling
d_62_ret["rank"] = d_62_ret.groupby(["date"]).cumcount()+1
d_62_ret = d_62_ret.sort_values(by = ["symbol", "date"])
d_62_ret["tag"] = np.where(d_62_ret["rank"] <= 100, 1, 0)
d_62_ret = d_62_ret.groupby(["symbol"]).rolling(3).agg({"tag": "sum", "date": "max"}).reset_index()
d_62_ret = d_62_ret[~ np.isnan(d_62_ret.tag)]
d_62_ret["date"] = d_62_ret["date"].astype("int")
d_62_ret[d_62_ret.tag == 3][["symbol", "date"]].sort_values(by = ["date"])

,symbol,date
190867,600228.SH,20120106
212668,600462.SH,20120109
190868,600228.SH,20120109
37327,000791.SZ,20120110
241335,600740.SH,20120111
...,...,...
110681,002475.SZ,20120629
237117,600702.SH,20120629
155159,300191.SZ,20120629
235479,600687.SH,20120629


63. 每天计算最近三天每天对行业的超额收益率都排进当天行业前30%的股票有哪些？

In [39]:
# 1. 每只股票的超额收益率
# 1.1 行业收益率
data["stkcd_ret"] = data["close"]/data["pre_close"] - 1
d_63_ret = data.groupby(["date", "industry"])["capt"].sum().reset_index().rename(columns = {"capt": "ind_capt"})
d_63_ret = data.merge(d_63_ret, left_on = ["date", "industry"], right_on = ["date", "industry"])
d_63_ret["capt_ratio"] = d_63_ret["capt"]/d_63_ret["ind_capt"]
d_63_ret["ratio_ret"] = d_63_ret["capt_ratio"]*d_63_ret["stkcd_ret"]
d_63_ret = d_63_ret.groupby(["date", "industry"])["ratio_ret"].sum().reset_index().rename(columns = {"ratio_ret": "ind_ret"})

d_63_ret = data[["date", "symbol", "stkcd_ret", "industry"]].merge(d_63_ret, how = "left", right_on = ["date", "industry"], left_on = ["date", "industry"])

# 1.2 循环回归
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

d_63_ret = d_63_ret.merge(d_63_ret.groupby("symbol").apply(regress, "stkcd_ret", ["ind_ret"]).reset_index().rename(columns = {"ind_ret" : 'beta'}), right_on = ['symbol'], left_on = ['symbol'])
d_63_ret["abnr_ret"] = d_63_ret["stkcd_ret"] - d_63_ret["ind_ret"]*d_63_ret["beta"] - d_63_ret["intercept"]
d_63_ret = d_63_ret[['date', 'symbol', 'industry', 'abnr_ret']].sort_values(by = ["date", "industry", "abnr_ret"], ascending = [True, True, False])

# 2. 设定rolling tag
d_63_ret["rank"] = d_63_ret.groupby(["industry", "date"]).cumcount() + 1
d_63_ret = d_63_ret.merge(
    d_63_ret.groupby(["industry", "date"])["rank"].agg(lambda x: x.max()*0.3).reset_index().rename(columns = {"rank": "rank_30p"}),
    right_on = ["industry", "date"],
    left_on = ["industry", "date"]
    )
d_63_ret = d_63_ret.sort_values(["symbol", "date"])
d_63_ret["rank"] = d_63_ret["rank"].astype(float)
d_63_ret["tag"] = np.where(d_63_ret["rank"] <= d_63_ret["rank_30p"], 1, 0)

d_63_ret= d_63_ret.groupby(["symbol"]).rolling(3).agg({"tag": "sum", "date": "max"}).reset_index()

d_63_ret = d_63_ret[~ np.isnan(d_63_ret.tag)]
d_63_ret["date"] = d_63_ret["date"].astype("int")
d_63_ret[d_63_ret.tag == 3][["symbol", "date"]].sort_values(by = ["date"])

,symbol,date
248469,600807.SH,20120106
215943,600499.SH,20120106
30071,000701.SZ,20120106
236652,600698.SH,20120106
250692,600827.SH,20120106
...,...,...
143108,300088.SZ,20120629
77219,002188.SZ,20120629
166881,300293.SZ,20120629
166142,300285.SZ,20120629
